In [ ]:
from stoneforge.io.lasparser import parse_las3
from stoneforge.io.las2 import read
from stoneforge.io.csv import read_csv

import os
import numpy as np
import pandas as pd
from pathlib import Path

In [ ]:
class WellProject:
    
    def __init__(self):
        self.wells = {}
    
    def add_well(self, name):
        """ Add a well to the project."""
        
        self.wells[name] = {
            'data' : {},
            'units' : {},
            'header' : {},
            'extras' : {}
        }
    
    def import_file(self, filepath, filetype=False, las_type='las2', **kwargs):
        """
        Import a file into the project.
        
        Parameters
        ----------
        - las_type: str, type of LAS file ('las2' or 'las3
        - filepath: str, path to the file to import
        - filetype: str, type of the file (e.g., 'las3', 'las2', etc.)
        
        Returns
        -------
        - Depending on the file type, it will return:
        """
        filepath = os.path.normpath(filepath)
        if not filetype:
            filetype = os.path.splitext(filepath)[-1].lower()
            
        filetypes = ['.las', '.csv', '.txt', '.dlis']
        if filetype not in filetypes:
            raise ValueError(f"Unsupported file type: {filetype}. Supported types are: {filetypes}")
            
        if filetype == '.las':
            if las_type == 'las2':
                return self._las2_micro_management(read(filepath))
            if las_type == 'las3':
                return self._las3_micro_management(parse_las3(filepath))
                
    def _las3_micro_management(self, las_data):
        print(las_data.keys())
        return las_data
                
    def _las2_micro_management(self, las_data):
        """    Convert LAS2 data into a structured format for easier manipulation.
        
        Parameters
        ----------
        - las_data: dict, parsed LAS2 data
        
        Returns
        -------
        - data: dict, structured data with keys 'curve', 'data', and other metadata
        """
        
        header = {}
        for key in las_data:
            if key not in ['data']:
                df = pd.DataFrame(las_data[key])
                header[key] = df
            data_header = list(las_data['curve'])
            
        data_header = list(header['curve']['mnemonic'])
        data = pd.DataFrame(las_data['data'].T, columns=data_header)
        
        units = header['curve'].set_index('mnemonic').T
        del header['curve']
                
        return {'header':header,'data':data,'units':units,}

In [117]:
# las3 example usage

ddt = WellProject()
a = ddt.import_file(r"../stoneforge/datasets/example_las3.las", las_type='las3')
print(a)

dict_keys(['VERSION INFORMATION', 'Well Information', 'Log_Parameter', 'Log_Definition', 'Drilling_Definition', 'Drilling_Data | Drilling_Definition', 'Core_Definition', 'Core_Data[1] | Core_Definition', 'Core_Data[2] | Core_Definition', 'Inclinometry_Definition', 'Inclinometry_Data | Inclinometry_Definition', 'Test_Definition', 'Test_Data | Test_Definition', 'TOPS_Definition', 'TOPS_Data | TOPS_Definition', 'Perforations_Definition', 'Perforations_Data | Perforations_Definition', 'Log_Data | Log_Definition'])
{'VERSION INFORMATION':        0      1                                          2
0  VERS.    3.0       CWLS LOG ASCII STANDARD -VERSION 3.0
1  WRAP.     NO                    ONE LINE PER DEPTH STEP
2  DLM .  COMMA  DELIMITING CHARACTER BETWEEN DATA COLUMNS, 'Well Information':            0                         1                       2
0    STRT .M                 1670.0000       First Index Value
1    STOP .M                 1669.7500        Last Index Value
2    STEP .M  

In [104]:
# Las2 example usage

ddt = WellProject()
ddt.import_file(r"../stoneforge/datasets/DP1.las")['data']


DEPT     SP    ILD    ILM    LL8        GR   CALI   DRHO   RHOB  \
0       100.0 -999.0 -999.0 -999.0 -999.0   25.1383 -999.0 -999.0 -999.0   
1       100.5 -999.0 -999.0 -999.0 -999.0   24.7342 -999.0 -999.0 -999.0   
2       101.0 -999.0 -999.0 -999.0 -999.0   24.9611 -999.0 -999.0 -999.0   
3       101.5 -999.0 -999.0 -999.0 -999.0   25.1880 -999.0 -999.0 -999.0   
4       102.0 -999.0 -999.0 -999.0 -999.0   25.4148 -999.0 -999.0 -999.0   
...       ...    ...    ...    ...    ...       ...    ...    ...    ...   
15688  7944.0 -999.0 -999.0 -999.0 -999.0 -999.0000 -999.0 -999.0 -999.0   
15689  7944.5 -999.0 -999.0 -999.0 -999.0 -999.0000 -999.0 -999.0 -999.0   
15690  7945.0 -999.0 -999.0 -999.0 -999.0 -999.0000 -999.0 -999.0 -999.0   
15691  7945.5 -999.0 -999.0 -999.0 -999.0 -999.0000 -999.0 -999.0 -999.0   
15692  7946.0 -999.0 -999.0 -999.0 -999.0 -999.0000 -999.0 -999.0 -999.0   

        NPHI     DT  
0     -999.0 -999.0  
1     -999.0 -999.0  
2     -999.0 -999.0  
3     -999.0 -999.0  
4     -999.0 -999.0  
...      ...    ...  
15688 -999.0 -999.0  
15689 -999.0 -999.0  
15690 -999.0 -999.0  
15691 -999.0 -999.0  
15692 -999.0 -999.0  

[15693 rows x 11 columns]